In [1]:
import pandas as pd
import math
from Util_Functions.map import mapper
from Util_Functions.haversine import haversine
from Util_Functions.load_coordinates import load_coordinates
from Util_Functions.calculate_shortest_path import dijkstra
from Util_Functions.create_graph_from_csv import create_graph_from_csv
map_obj = mapper(True)
map_obj


ModuleNotFoundError: No module named 'folium'

In [ ]:

# Load coordinates
df = pd.read_csv('../assets/location_coordinates_final.csv')  # Assumes columns: name, Latitude, Longitude

# Initialize matrix
stops = df['name'].tolist()
matrix_data = []

# Calculate pairwise distances
for index1, row1 in df.iterrows():
    row_distances = {'name': row1['name']}
    for index2, row2 in df.iterrows():
        dist = 0 if index1 == index2 else haversine(
            row1['Latitude'], row1['Longitude'],
            row2['Latitude'], row2['Longitude']
        )
        row_distances[row2['name']] = round(dist, 3)
    matrix_data.append(row_distances)

# Convert to DataFrame and save
dist_matrix_df = pd.DataFrame(matrix_data)
dist_matrix_df.to_csv('../assets/distances_log.csv')

print("✅ distances_matrix.csv created with", len(stops), "x", len(stops), "matrix.")


✅ distances_matrix.csv created with 86 x 86 matrix.


In [ ]:
dist = pd.read_csv("../assets/distances_log.csv")

In [ ]:
dist.head()

,Unnamed: 0,name,clock tower,darshanlal chowk,saharanpur chowk,isbt dehradun,graphic era hill university,asley hall,matawala bagh,daudwala,...,panditwari,vasant vihar,selaqui chowk,baliwala chowk,bahuwala raja road,raja road,bansiwala,manduwala,prem nagar,balliwala chowk
0,0,clock tower,0.000,0.212,1.457,5.840,6.965,0.567,2.103,7.497,...,5.200,4.073,25.993,0.212,1.148,0.952,14.117,11.985,7.821,0.212
1,1,darshanlal chowk,0.212,0.000,1.419,5.746,6.835,0.676,2.067,7.310,...,5.289,4.142,26.114,0.000,0.946,0.843,14.212,12.157,7.909,0.000
2,2,saharanpur chowk,1.457,1.419,0.000,4.428,5.665,2.021,0.648,6.607,...,4.159,2.944,25.067,1.419,1.316,0.679,13.054,11.534,6.742,1.419
3,3,isbt dehradun,5.840,5.746,4.428,0.000,1.698,6.405,3.838,4.339,...,4.788,4.110,23.798,5.746,5.122,4.904,11.646,12.500,6.154,5.746
4,4,graphic era hill university,6.965,6.835,5.665,1.698,0.000,7.511,5.154,3.032,...,6.484,5.789,24.897,6.835,6.065,6.013,12.843,14.134,7.707,6.835


In [ ]:
# Build the graph
file_path = '../assets/distances_log.csv'
graph_obj = create_graph_from_csv(file_path)

# Display it (optional)
graph_obj.display_graph()


clock tower->darshanlal chowk (0.212),saharanpur chowk (1.457),isbt dehradun (5.84),graphic era hill university (6.965),asley hall (0.567),matawala bagh (2.103),daudwala (7.497),mathurawala (4.664),vishnupuram (4.606),bangali kothi (1.481),kargi chowk (4.353),ranipokhri (22.858),doiwala (18.273),lachiwala (16.683),kuanwala (1.006),harawala (5.875),miawala (5.045),mokampur (6.646),jogiwala (4.839),rispana (5.102),gujraunwala (1.006),raipur chowk (5.761),dobhal chowk (4.245),6 no. pulia (1.603),ring road (3.688),post office nehru gram (4.695),graphic era hill university/gehu (7.708),hathibadkala (3.642),garhi cant (3.383),vijay coloney (3.325),chir bagh (1.592),cm house (1.651),ongc chowk (2.571),ballupur chowk (0.953),gms road (2.781),rajender nagar (2.473),yamuna coloney (0.771),bindal pul (0.771),kishan nagar (0.771),blood bank (0.58),supply (6.93),it park dehradun (5.883),nala paani chowk (2.195),shastdhara crossing (1.901),ladpur (1.723),fountain chowk rispana (1.527),fountain chowk

In [ ]:
# Define start and end points
start_point = "clock tower"
end_point = "asley hall"

# Calculate shortest path
path, distance = dijkstra(graph_obj, start_point, end_point)

# Display results
if path is not None:
    print("📍 Path:", " -> ".join(path))
    print("📏 Distance:", round(distance, 2), "km")
else:
    print("No valid path exists between", start_point, "and", end_point)


NameError: name 'dijkstra' is not defined

In [ ]:
import folium
from Util_Functions.plot_map import plot_route_map



# Load coordinates and build coordinate dictionary
coord_dict, coords = load_coordinates("../assets/location_coordinates_final.csv")

# Create the graph from distance CSV
graph_obj = create_graph_from_csv("../assets/distances_log.csv")

# Plot the map
m = plot_route_map(coord_dict, graph_obj, start_point, end_point)

# Display the map
m




In [ ]:
from Util_Functions.knn import nearest
from Util_Functions.plot_map import plot_route_map
from Util_Functions.map import selective
from Util_Functions.plot_map import selective_line

nearby = nearest("baliwala chowk", k=10)
list_nearest = []
for station, dist in nearby:
    list_nearest.append(station)
    print(f"{station}: {dist:.2f} km")

m = selective_line(coord_dict, graph_obj, list_nearest)
m





darshanlal chowk: 0.00 km
balliwala chowk: 0.00 km
clock tower: 0.21 km
tehsil: 0.24 km
great value: 0.34 km
blood bank: 0.38 km
asley hall: 0.68 km
ballupur chowk: 0.78 km
raja road: 0.84 km
race course: 0.90 km


In [ ]:
from Util_Functions.schedule import get_bus_details
get_bus_details("clock tower")


🚏 Bus Details for Stop: Clock Tower
[Arjun Gurung, Bus 9, On Stop, ['7:00 AM'], Next: Darshanlal Chowk, ['7:15 AM']]
[Sunil Kumar IV, Bus 18, On Stop, ['7:00 AM'], Next: Darshanlal Chowk, ['7:15 AM']]
[M.R Rama, Bus 24, On Stop, ['7:00 AM'], Next: Darshanlal Chowk, ['7:15 AM']]
[Gudmohan, Bus 42, On Stop, ['7:15 AM'], Next: Darshanlal Chowk, ['7:00 AM']]
[Hari Singh, Bus 64, On Stop, ['7:15 AM'], Next: Darshanlal Chowk, ['7:00 AM']]
[Gudmohan, Bus 42, Incoming, ['7:00 AM'], Next: Clock Tower, ['Will reach at this time']]
[Hari Singh, Bus 64, Incoming, ['7:00 AM'], Next: Clock Tower, ['Will reach at this time']]
